In [39]:
import pandas as pd
from datetime import datetime

In [40]:
df = pd.read_csv('calc_logs.csv')
display(df)

,timestamp,key,input_type
0,2021-05-22 17:51:27.063842,5,KEYBOARD
1,2021-05-22 17:51:27.184179,6,KEYBOARD
2,2021-05-22 17:51:27.758680,*,KEYBOARD
3,2021-05-22 17:51:28.437248,4,KEYBOARD
4,2021-05-22 17:51:28.723428,2,KEYBOARD
...,...,...,...
87,2021-05-22 17:52:57.338106,5,BUTTON
88,2021-05-22 17:52:57.524092,Evaluate,KEYBOARD
89,2021-05-22 17:53:00.112729,+,BUTTON
90,2021-05-22 17:53:00.663068,3,BUTTON


In [42]:
# here we separate all inputs from the csv into the "blocks" in which they happened
# this is done by grouping all inputs with a delay of less than 5 seconds together

input_blocks = [] # list of lists - holds the separate chains of inputs
last_time = datetime.min
start_row = None
current_list = []

for row in df.iterrows():
    current_time = row[1]['timestamp']
    
    if start_row == None:
        start_row = row
        
    current_time = datetime.strptime(current_time, "%Y-%m-%d %H:%M:%S.%f")
        
    timedelta = current_time - last_time
    
    if timedelta.seconds > 5:
        input_blocks.append(current_list.copy())
        current_list.clear()
        current_list.append(row)
        
    else:
        current_list.append(row)
        
    last_time = current_time

3


Time to get some details

In [58]:
last_timestamp = None
last_row = None
hover_delta_list = []

for block in input_blocks:
    for row in block:
        if last_timestamp == None:
            last_timestamp = datetime.strptime(row[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f")
            last_row = row
            continue
            
        else:
            current_timestamp = datetime.strptime(row[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f")
            
            if last_row[1]['input_type'] != row[1]['input_type']:
                hover_delta_list.append(current_timestamp - last_timestamp)
            
            last_timestamp = current_timestamp
    
    last_timestamp = None
    last_row = None
  
print("Hovering times between mouse and keyboard in milliseconds:")
for delta in hover_delta_list:
    print(delta.total_seconds() * 1000)

Hovering times between mouse and keyboard in milliseconds:
1332.664
668.396
547.197
445.941
952.203
1663.791
419.35
382.747
1052.072
462.409
716.037
